# QnA Matching Tutorial (Prepare Train and Test Data)

## Overview

Question answering systems of specific topics are highly demanded but are not quite available yet. The common use cases we see in this type of scenario include but are not limited to:
* Live chat support
* Chat bot
* Document match - find a subcategory of financial/legal/.. documents that answers a particular question

Therefore, we have provided 2 Notebooks with step-by-step descriptions of how to match the correct answer to a given question. To solve this problem, we train the classification models on a set of pre-canned question-answer pairs and classify a new question to its correct answer class.

## Import Required Python Modules

In this notebook, we use several open-source Python packages that need to be installed in a local machine or an Azure Notebook Server. An upgrade is requested if a previous version of a package has been installed in the past.

We make use of the NLTK sentence tokenization capability which takes a long string of text and splits it into sentence units. The tokenizer requires the installation of the 'punkt' tokenizer models. After importing nltk, the nltk.download() function can be used to download specific packages such as 'punkt'.

In [1]:
# uncomment the below code to install/upgrade the requested Python packages.
# !pip install --upgrade --no-deps smart_open azure pandas nltk

In [2]:
import pandas as pd
import numpy as np
import re, sys, os, gc, requests, time, math, nltk, glob, os.path, ftplib, json, base64, datetime, warnings, gzip
from collections import (namedtuple, Counter)
from azure.storage import CloudStorageAccount
warnings.filterwarnings("ignore")

In [3]:
EMPTY = ''

## Access Sample Data

We use three sets of data in this series of notebooks. We collect the raw data from the Stack Overflow Database and extract all question-answer pairs related to the __"JavaScript"__ tag. For the question-answer pairs, we consider the following scenarios.

1. Original Questions (Q): These questions have been asked and answered on the Stack Overflow.
2. Duplications (D): There is a linkage among the questions. Some questions that have already been asked by others are linked to the previous/original questions as Duplications. In the Stack Overflow Database, this kind of linkage is determined by "LINK_TYPE_DUPE = 3". Each original question could have 0 to many duplications, which are considered as semantically equivalent to the original question.
3. Answers (A): For each Original question and its Duplications, we have found more than one answers have solved that question. In our analysis, we only select the Accepted answer or the answer with the highest score that solved the Original question. Therefore, it's 1-to-1 mapping between Original questions and Answers and many-to-1 mapping between Duplications and Original questions. Each Original question and its Duplications have an unique AnswerId.
4. Function Words: we consider a list of words that can only be used in between content words in the creation of phrases. This list of words, stored as a .txt file, is also used as Stop Words.

See the below Data Diagram to illustrate the relationship among Original Questions (Q), Duplications (D) and Answers (A):

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/data_diagram.png">

The data schema is:

| Dataset | Column Name | Description
| ----------|------------|--------
| questions | Id | the unique question ID (primary key)
|  | AnswerId | the unique answer ID per question
|  | Text0 | the raw text data including the question's title and body
|  | CreationDate | the timestamp of when the question has been asked
| dupes | Id | the unique duplication ID (primary key)
|  | AnswerId | the answer ID associated with the duplication
|  | Text0 | the raw text data including the duplication's title and body
|  | CreationDate | the timestamp of when the duplication has been asked
| answers | Id | the unique answer ID (primary key)
|  | text0 | the raw text data of the answer


In [4]:
# functions to load .tsv.gz file into Pandas data frame.
def read_csv_gz(url, **kwargs):
    return pd.read_csv(gzip.open(requests.get(url, stream=True).raw, mode='rb'), **kwargs)

def read_data_frame(url, **kwargs):
    return read_csv_gz(url, sep='\t', encoding='utf8', **kwargs).set_index('Id')

In [5]:
# URLs to Original questions, Duplications, Answers and Function Words.
questions_url = 'https://mezsa.blob.core.windows.net/stackoverflow/orig-q.tsv.gz'
dupes_url = 'https://mezsa.blob.core.windows.net/stackoverflow/dup-q.tsv.gz'
answers_url = 'https://mezsa.blob.core.windows.net/stackoverflow/ans.tsv.gz'

In [6]:
# load datasets.
questions = read_data_frame(questions_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
dupes = read_data_frame(dupes_url, names=('Id', 'AnswerId', 'Text0', 'CreationDate'))
answers = read_data_frame(answers_url, names=('Id', 'Text0'))

## Text Pre-processing

### Clean up text

Since the raw data is in HTML format, we need to clean up HTML tags and links. We also remove embedded code chunks.

In [7]:
def strip_code(text):
    if not isinstance(text, str): return text
    return re.sub('<pre><code>.*?</code></pre>', EMPTY, text)

def strip_tags(text):
    if not isinstance(text, str): return text
    return re.sub('<[^>]+>', EMPTY, text)

def strip_links(text):
    if not isinstance(text, str): return text
    def replace_link(match):
        return EMPTY if re.match('[a-z]+://', match.group(1)) else match.group(1)
    return re.sub('<a[^>]+>(.*)</a>', replace_link, text)

def clean_text(text):
    return strip_tags(strip_links(strip_code(text)))

In [8]:
for df in (questions, dupes, answers):
    df['Text'] = df['Text0'].apply(clean_text).str.lower()
    df['NumChars'] = df['Text'].str.len()

### Set data selection criteria

To obtain the high quality datasets for learning phrases, we set a threshold of minimum length of characters in the text field. This threshold is considered respectively for Original questions, Duplications and Answers. 

For each Original question, we also make sure there are at least 3 linked Duplications so that we have enough data to learn from in the later Notebooks.

In [9]:
# a function to find the AnswerIds has at least 3 dupes.
def find_answerId(answersC, dupesC, num_dupes):
       
    countHash = {}
    for i in dupesC.AnswerId:
        if i not in answersC.index.values:
            continue
        if i not in countHash.keys():
            countHash[i] = 1
        else:
            countHash[i] += 1
            
    countHash = {k: v for k, v in countHash.items() if v >= num_dupes}
    commonAnswerId = countHash.keys()
    
    return commonAnswerId

# a function to extract data based on the selection criteria.
def select_data(questions, dupes, answers):
    # exclude the records without any text
    questions_nz = questions.query('NumChars > 0')
    dupes_nz = dupes.query('NumChars > 0')
    answers_nz = answers.query('NumChars > 0')

    # get the 10th percentile of text length as the minimum length of characters to consider in the text field
    minLenQ = questions_nz.quantile(.1)['NumChars']
    minLenD = dupes_nz.quantile(.1)['NumChars']
    minLenA = answers_nz.quantile(.1)['NumChars']
    
    # eliminate records with text less than the minimum length
    questionsC = questions.query('NumChars >' + str(int(minLenQ)))
    dupesC = dupes.query('NumChars >' + str(minLenD))
    answersC = answers.query('NumChars >' + str(minLenA))
    
    # remove the records in dupesC whose questionId has already existed in questionsC
    duplicatedIndex = list(set(questionsC.index).intersection(set(dupesC.index)))
    dupesC.drop(duplicatedIndex, inplace=True)
    
    # make sure Questions 1:1 match with Answers 
    matches = questionsC.merge(answersC, left_on = 'AnswerId', right_index = True)
    questionsC = matches[['AnswerId', 'Text0_x', 'CreationDate', 'Text_x', 'NumChars_x']]
    questionsC.columns = ['AnswerId', 'Text0', 'CreationDate', 'Text', 'NumChars']

    answersC = matches[['Text0_y', 'Text_y', 'NumChars_y']]
    answersC.index = matches['AnswerId']
    answersC.columns = ['Text0', 'Text', 'NumChars']
    
    # find the AnswerIds has at least 3 dupes
    commonAnswerId = find_answerId(answersC, dupesC, 3)
    
    # select the records with those AnswerIds
    questionsC = questionsC.loc[questionsC.AnswerId.isin(commonAnswerId)]
    dupesC = dupesC.loc[dupesC.AnswerId.isin(commonAnswerId)]
    answersC = answersC.loc[commonAnswerId] 
    
    return questionsC, dupesC, answersC

In [10]:
# some questions have been linked to multiple answerIds.
# therefore, remove the duplicated answerId for those questions.
questions = questions.groupby(questions.index).first()
dupes = dupes.groupby(dupes.index).first()
questionsC, dupesC, answersC = select_data(questions, dupes, answers)

## Prepare Training and Test datasets

We split questions based on the creation date so that the training and the test sets are defined as below.
1. training set = Original questions + 75% of oldest Duplications per Original question
2. test set = remaining 25% of Duplications per Original question

In [11]:
# a function to split Original questions and their Duplications into training and test sets.
def split_data(questions, dupes, frac):
    trainQ = questions
    testQ = pd.DataFrame(columns = dupes.columns.values) # create an empty data frame
    for answerId in np.unique(dupes.AnswerId):
        df = dupes.query('AnswerId == ' + str(answerId))
        totalCount = len(df)
        splitPoint = int(totalCount * frac)
        dfSort = df.sort_values(by = ['CreationDate'])
        trainQ = trainQ.append(dfSort.head(splitPoint)) # oldest N percent of duplications
        testQ = testQ.append(dfSort.tail(totalCount - splitPoint))
    
    # convert data type to int
    testQ[["AnswerId", "NumChars"]] = testQ[["AnswerId", "NumChars"]].astype(int) 
    # rename the index 
    testQ.index.rename("Id", inplace=True)
    
    return trainQ, testQ

In [12]:
# prepare training and test
trainQ, testQ = split_data(questionsC, dupesC, 0.75)

## Select Subsets with Sufficient Training Questions per Answer Class

In our past experiment, we have noticed that some Answer class only contains a small number of training example. Training a classifier on a small amount of examples is not sufficient. Therefore, we have performed an analysis to study how the size of training example per class actually impact on the model performance.

In this study, we test the __Average Rank__ and __Top 10 Percentage__ (two evaluation measures) distribution with different numbers of training examples per class. As we can see from the distribution below, our ensemble model can secure an __Average Rank__ less than 20 (out of 1201 different answer classes) and a __Top 10 Percentage__ over 70% when we have more than 15 training examples per class.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/training_size.PNG">

Even the number of classes that have more than 15 training examples are very limited in our particular example, but this study is very meaningful for future works as we have learned the training example size is very critical and having a decent number of training examples per class is a must have.

<img src="https://raw.githubusercontent.com/Azure/Document_Matching/master/pic/training_size_details.PNG">

With the above study, we have decided to only consider the answer classes that have more than 13 training examples in this tutorial that reduces the entire dataset to 5,153 training examples, 1,735 test examples, and 103 unique answer classes.

In [13]:
countPerAns = pd.DataFrame({"NumTrain" : trainQ.groupby("AnswerId").size()})
trainQwithCount = trainQ.merge(countPerAns, left_on="AnswerId", right_index=True)
testQwithCount = testQ.merge(countPerAns, left_on="AnswerId", right_index=True)
answersCwithCount = answersC.merge(countPerAns, left_index=True, right_index=True)

# for each Answer class, we request more than 13 training examples.
trainQ_sub = trainQwithCount[trainQwithCount["NumTrain"] > 13]
testQ_sub = testQwithCount[testQwithCount["NumTrain"] > 13]
answersC_sub = answersCwithCount[answersCwithCount["NumTrain"] > 13]

In [14]:
trainQ[["AnswerId", "Text"]].head(5)

,AnswerId,Text
Id,,
3713,3777,call asp.net function from javascript?. i'm wr...
5223,6700,"length of a javascript object (that is, associ..."
7477,7523,autosizing textarea using prototype. i'm curre...
18082,1830844,validate decimal numbers in javascript - isnum...
21294,242607,dynamically load a javascript file. how can yo...


## Save cleaned data as .tsv and upload to Azure Blob

In [15]:
# Configure Blob Storage
storage_account_name = 'mezsa'
storage_account_key = 'X1Xwyn5ROxyQa4tmvjSza/Lv5bXLu7cZ1jWyfFhCEBCKFr78onDgFUH05F5iG2aq1IsU+DIooYDbPzKa821FSA=='
account = CloudStorageAccount(account_name=storage_account_name, account_key=storage_account_key)
blob_service = account.create_blob_service()

def save_upload_data(data, file_path, container_name, blob_name):
    data.to_csv(file_path, sep='\t', header=True, index=True, index_label='Id')
    blob_service.put_block_blob_from_path(container_name=container_name, blob_name=blob_name, file_path=file_path)

In [16]:
# modify the path in below script to upload the datasets to your own Blob Storage.
if False: 
    save_upload_data(trainQ_sub, os.path.join(os.getcwd(), "trainQ_tutorial.tsv"), 'stackoverflownew', 'trainQ_tutorial.tsv')
    save_upload_data(testQ_sub, os.path.join(os.getcwd(), "testQ_tutorial.tsv"), 'stackoverflownew', 'testQ_tutorial.tsv')
    save_upload_data(answersC_sub, os.path.join(os.getcwd(), "answersC_tutorial.tsv"), 'stackoverflownew', 'answersC_tutorial.tsv')